# Comparison of Immigrant Demographics: United States and Germany
## Data Cleaning, Wrangling, and Transformation

In [1]:
from IPython.display import display, HTML

## Introduction 

As part of my Masters program in Political Economy, I wrote a research paper exploring how different forms of "entry categories" in the United States (U.S.) and Germany affect immigrants' access to social welfare programs. You can read that paper, "The Life of a Civilized Being: A comparative analysis of immigrant social rights within the United States and Germany", <a href="https://wordpress.com/post/theresagasinski.wordpress.com/2176">here</a>. 

To complement that research, I designed a few interactive graphics to explore recent demographic developments in the U.S. and Germany using Plotly. This notebook contains the data cleaning, wrangling and transformation that was required before the plots could be rendered. To see my final graphs, click <a href="https://nbviewer.jupyter.org/gist/TGasinski/ec2eaf17b8ebc3d11f5d78ea7055ca29">here</a>. 

## Data Sources

__US Datasets Retrieved From:__ US Office of Immigration Statistics, US Census Bureau, and Migration Policy Institute


__German Datasets Retreived From:__ Statistiches Bundesamt

In [2]:
# import packages

import pandas as pd
import csv as csv
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns

import plotly.plotly as py
import plotly.graph_objs as go
from plotly import tools
from plotly.offline import init_notebook_mode, iplot

pd.options.mode.chained_assignment = None

In [3]:
init_notebook_mode(connected=True)

In [4]:
# read files

# us legal permanent residents
us_lpr_number = pd.read_excel(r'C:\Users\t_gas\Desktop\Political Economy - Masters\WS\Work, Labor, Social Reproduction\Term Paper\Data\fy2017_table1.xlsx')
us_lpr_country_17 = pd.read_excel(r'C:\Users\t_gas\Desktop\Political Economy - Masters\WS\Work, Labor, Social Reproduction\Term Paper\Data\fy2017_immsuptable1d.xlsx')

# us other nonimmigrants
us_nonimm_number_1 = pd.read_excel(r'C:\Users\t_gas\Desktop\Political Economy - Masters\WS\Work, Labor, Social Reproduction\Term Paper\Data\Table 25d.xls')
us_nonimm_number_2 = pd.read_excel(r'C:\Users\t_gas\Desktop\Political Economy - Masters\WS\Work, Labor, Social Reproduction\Term Paper\Data\fy2017_table25d.xlsx')
us_nonimm_country_17 = pd.read_excel(r'C:\Users\t_gas\Desktop\Political Economy - Masters\WS\Work, Labor, Social Reproduction\Term Paper\Data\fy2017_nonimmsuptable1d.xlsx')

# us refugees and asylum seekers
us_ref_asy_number = pd.read_excel(r'C:\Users\t_gas\Desktop\Political Economy - Masters\WS\Work, Labor, Social Reproduction\Term Paper\Data\MPI-Data-Hub_Refugee-Admissions_2018.xlsx') 

# us population 
us_pop_1 = pd.read_excel(r'C:\Users\t_gas\Desktop\Political Economy - Masters\WS\Work, Labor, Social Reproduction\Term Paper\Data\st-est00int-01.xls')
us_pop_2 = pd.read_csv(r'C:\Users\t_gas\Desktop\Political Economy - Masters\WS\Work, Labor, Social Reproduction\Term Paper\Data\nst-est2018-alldata.csv')

# germany immigration data (number, country of origin)
de_imm_number = pd.read_excel(r'C:\Users\t_gas\Desktop\Political Economy - Masters\WS\Work, Labor, Social Reproduction\Term Paper\Data\Germany_1998-2017_Ausländer_Numbers.xls')
de_imm_country_17 = pd.read_excel(r'C:\Users\t_gas\Desktop\Political Economy - Masters\WS\Work, Labor, Social Reproduction\Term Paper\Data\Germany_2017_Ausländer.xls')

# germany refugee, asylum seekers, and humanitarian grounds
de_ref_asy_hum_number = pd.read_excel(r'C:\Users\t_gas\Desktop\Political Economy - Masters\WS\Work, Labor, Social Reproduction\Term Paper\Data\Germany_Refugees_Asylees_Year.xls')

# germany population data
de_pop = pd.read_excel(r'C:\Users\t_gas\Desktop\Political Economy - Masters\WS\Work, Labor, Social Reproduction\Term Paper\Data\Germany_Population.xls')

## U.S. Data

In [5]:
# fix layout

us_lpr_number = us_lpr_number[3:]
lpr1 = us_lpr_number[['Table 1.', 'Unnamed: 1']].copy()
lpr1.rename(columns = {'Table 1.': 'Year', 'Unnamed: 1': 'Number'}, inplace=True)
lpr2 = us_lpr_number[['Unnamed: 2', 'Unnamed: 3']].copy()
lpr2.rename(columns = {'Unnamed: 2': 'Year', 'Unnamed: 3': 'Number'}, inplace=True)
lpr3 = us_lpr_number[['Unnamed: 4', 'Unnamed: 5']].copy()
lpr3.rename(columns = {'Unnamed: 4': 'Year', 'Unnamed: 5': 'Number'}, inplace=True)
lpr4 = us_lpr_number[['Unnamed: 6', 'Unnamed: 7']].copy()
lpr4.rename(columns = {'Unnamed: 6': 'Year', 'Unnamed: 7': 'Number'}, inplace=True)
lpr5 = lpr1.append(lpr2, ignore_index=True)
lpr6 = lpr3.append(lpr4, ignore_index=True)
us_lpr = lpr5.append(lpr6, ignore_index=True)
us_lpr.rename(columns = {'Number': 'Number LPR'}, inplace = True)

In [6]:
# convert to string

us_lpr['Year'] = us_lpr['Year'].astype(str)

In [7]:
# fix layout (only keep labor migrant data)

headers_1 = us_nonimm_number_1.iloc[2]
us_nonimm_number_1 = us_nonimm_number_1.iloc[3:83]
us_nonimm_number_1.rename(columns = headers_1, inplace=True)
us_nonimm_1 = us_nonimm_number_1.transpose()
headers_2 = us_nonimm_1.iloc[0]
us_nonimm_1 = us_nonimm_1.iloc[1:]
us_nonimm_1.rename(columns = headers_2, inplace=True)
us_nonimm_1.rename(columns=lambda x: x.strip(), inplace = True)

In [8]:
# add new column with total numbers

us_nonimm_1 = us_nonimm_1[['Total All Admissions 1', 'Total I-94 Admissions', 'Temporary workers and families', 'Temporary workers and trainees', 'Temporary workers with “specialty occupation” (H1B)', 'Chile and Singapore Free Trade Agreement aliens (H1B1)', 'Registered nurses participating in the Nursing Relief for Disadvantaged Areas (H1C)', 'Seasonal agricultural workers (H2A) 4', 'Seasonal nonagricultural workers (H2B) 4', 'Returning H2B workers (H2R) 4', 'Trainees (H3)', 'Spouses and children of H1, H2, or H3 (H4)', 'Workers with extraordinary ability or achievement (O1)', 'Workers accompanying and assisting in performance of O1 workers (O2)', 'Spouses and children of O1 and O2 (O3)']].copy()
us_nonimm_1.loc[us_nonimm_1['Chile and Singapore Free Trade Agreement aliens (H1B1)'] == 'X', 'Chile and Singapore Free Trade Agreement aliens (H1B1)'] = 0
us_nonimm_1.loc[us_nonimm_1['Registered nurses participating in the Nursing Relief for Disadvantaged Areas (H1C)'] == 'X', 'Registered nurses participating in the Nursing Relief for Disadvantaged Areas (H1C)'] = 0
us_nonimm_1.loc[us_nonimm_1['Registered nurses participating in the Nursing Relief for Disadvantaged Areas (H1C)'] == '-', 'Registered nurses participating in the Nursing Relief for Disadvantaged Areas (H1C)'] = 0
us_nonimm_1.loc[us_nonimm_1['Returning H2B workers (H2R) 4'] == 'X', 'Returning H2B workers (H2R) 4'] = 0
columns_1 = list(us_nonimm_1)
columns_1.remove('Total All Admissions 1')
columns_1.remove('Total I-94 Admissions')
us_nonimm_1['Labor Migrants'] = us_nonimm_1[columns_1].sum(axis=1)
us_labor_1 = us_nonimm_1[['Labor Migrants']].copy()
us_labor_1.rename(columns = {'index' : 'Year'}, inplace = True)

In [9]:
# fix layout

headers_3 = us_nonimm_number_2.iloc[2]
us_nonimm_number_2 = us_nonimm_number_2.iloc[3:85]
us_nonimm_number_2.rename(columns = headers_3, inplace=True)
us_nonimm_2 = us_nonimm_number_2.transpose()
headers_4 = us_nonimm_2.iloc[0]
us_nonimm_2 = us_nonimm_2.iloc[1:]
us_nonimm_2.rename(columns = headers_4, inplace=True)
us_nonimm_2.rename(columns=lambda x: x.strip(), inplace = True)

In [10]:
# add new column with total numbers

us_nonimm_2 = us_nonimm_2[['Total all admissions 2', 'Total I-94 admissions 3', 'Temporary workers and families', 'Temporary workers and trainees', 'CNMI-only transitional workers (CW1)', 'Spouses and children of CW1 (CW2)', 'Temporary workers in specialty occupations (H1B)', 'Chile and Singapore Free Trade Agreement aliens (H1B1)', 'Registered nurses participating in the Nursing Relief for Disadvantaged Areas (H1C)', 'Agricultural workers (H2A) 4', 'Nonagricultural workers (H2B)', 'Returning H2B workers (H2R) 5', 'Trainees (H3)', 'Spouses and children of H1, H2, or H3 (H4)', 'Workers with extraordinary ability or achievement (O1)', 'Workers accompanying and assisting in performance of O1 workers (O2)', 'Spouses and children of O1 and O2 (O3)', 'Internationally recognized athletes or entertainers (P1)', 'Artists or entertainers in reciprocal exchange programs (P2)', 'Artists or entertainers in culturally unique programs (P3)', 'Spouses and children of P1, P2, or P3 (P4)', 'Workers in international cultural exchange programs (Q1)', 'Workers in religious occupations (R1)', 'Spouses and children of R1 (R2)', 'North American Free Trade Agreement (NAFTA) professional workers (TN)', 'Spouses and children of TN (TD)', 'Intracompany transferees', 'Intracompany transferees (L1)', 'Spouses and children of L1 (L2)', 'Treaty traders and investors', 'Treaty traders and their spouses and children (E1)', 'Treaty investors and their spouses and children (E2)', 'Treaty investors and their spouses and children (CNMI only) (E2C)', 'Australian Free Trade Agreement principals, spouses and children (E3)', 'Representatives of foreign information media', 'Representatives of foreign information media and spouses and children (I1)']].copy()
us_nonimm_2.loc[us_nonimm_2['CNMI-only transitional workers (CW1)'] == 'X', 'CNMI-only transitional workers (CW1)'] = 0
us_nonimm_2.loc[us_nonimm_2['CNMI-only transitional workers (CW1)'] == 'D', 'CNMI-only transitional workers (CW1)'] = 0
us_nonimm_2.loc[us_nonimm_2['CNMI-only transitional workers (CW1)'] == '-', 'CNMI-only transitional workers (CW1)'] = 0
us_nonimm_2.loc[us_nonimm_2['Spouses and children of CW1 (CW2)'] == 'X', 'Spouses and children of CW1 (CW2)'] = 0
us_nonimm_2.loc[us_nonimm_2['Spouses and children of CW1 (CW2)'] == '-', 'Spouses and children of CW1 (CW2)'] = 0
us_nonimm_2.loc[us_nonimm_2['Chile and Singapore Free Trade Agreement aliens (H1B1)'] == 'D', 'Chile and Singapore Free Trade Agreement aliens (H1B1)'] = 0
us_nonimm_2.loc[us_nonimm_2['Registered nurses participating in the Nursing Relief for Disadvantaged Areas (H1C)'] == 'D', 'Registered nurses participating in the Nursing Relief for Disadvantaged Areas (H1C)'] = 0
us_nonimm_2.loc[us_nonimm_2['Registered nurses participating in the Nursing Relief for Disadvantaged Areas (H1C)'] == '-', 'Registered nurses participating in the Nursing Relief for Disadvantaged Areas (H1C)'] = 0
us_nonimm_2.loc[us_nonimm_2['Returning H2B workers (H2R) 5'] == 'D', 'Returning H2B workers (H2R) 5'] = 0
us_nonimm_2.loc[us_nonimm_2['Treaty investors and their spouses and children (CNMI only) (E2C)'] == 'X', 'Treaty investors and their spouses and children (CNMI only) (E2C)'] = 0
us_nonimm_2.loc[us_nonimm_2['Treaty investors and their spouses and children (CNMI only) (E2C)'] == 'D', 'Treaty investors and their spouses and children (CNMI only) (E2C)'] = 0
us_nonimm_2.loc[us_nonimm_2['Treaty investors and their spouses and children (CNMI only) (E2C)'] == '-', 'Treaty investors and their spouses and children (CNMI only) (E2C)'] = 0
us_nonimm_2.loc[us_nonimm_2['Australian Free Trade Agreement principals, spouses and children (E3)'] == 'D', 'Australian Free Trade Agreement principals, spouses and children (E3)'] = 0
columns_2 = list(us_nonimm_2)
columns_2.remove('Total all admissions 2')
columns_2.remove('Total I-94 admissions 3')
us_nonimm_2['Labor Migrants'] = us_nonimm_2[columns_2].sum(axis=1)
us_labor_2 = us_nonimm_2[['Labor Migrants']].copy()
us_labor_2.rename(columns = {'index' : 'Year'}, inplace = True)

In [11]:
# merge labor migrant data

us_labor = us_labor_1.append(us_labor_2)
us_labor = us_labor.reset_index()
us_labor.rename(columns={'index' : 'Year'}, inplace=True)
us_labor.loc[us_labor['Year'] == '2016 1', 'Year'] = '2016'
us_labor['Year'] = us_labor['Year'].astype(str)

In [12]:
# fix layout

headers_5 = us_ref_asy_number.iloc[5]
us_ref_asy_number_small = us_ref_asy_number.iloc[6:45]
#us_ref_asy_number_small = us_ref_asy_number.iloc[6:]
us_ref_asy_number_small.rename(columns=headers_5, inplace=True)
us_ref_asy = us_ref_asy_number_small.iloc[20:45]
us_ref_asy['Year'] = us_ref_asy['Year'].astype(str)

In [13]:
# fix layout

headers_6 = us_pop_1.iloc[2]
us_pop_1 = us_pop_1.drop(['Unnamed: 2', 'Unnamed: 12', 'Unnamed: 13'], axis=1)
us_pop_1 = us_pop_1.iloc[:4]
us_pop_1 = us_pop_1.iloc[3:]
us_pop_1.rename(columns = {'Unnamed: 1' : '2000'}, inplace=True)
us_pop_1.rename(columns = headers_6, inplace=True)
us_pop_1 = us_pop_1.transpose()
us_pop_1 = us_pop_1.reset_index()
us_pop_1.rename(columns = {'index' : 'Year', 3 : 'United States'}, inplace=True)
us_pop_1 = us_pop_1.iloc[1:]

In [14]:
# fix layout

us_pop_2 = us_pop_2[['NAME', 'CENSUS2010POP', 'POPESTIMATE2011', 'POPESTIMATE2012', 'POPESTIMATE2013', 'POPESTIMATE2014', 'POPESTIMATE2015', 'POPESTIMATE2016', 'POPESTIMATE2017', 'POPESTIMATE2018']].copy()
us_pop_2 = us_pop_2.iloc[:1]
us_pop_2_dict = {'CENSUS2010POP' : '2010', 'POPESTIMATE2011': '2011', 'POPESTIMATE2012' : '2012', 'POPESTIMATE2013' : '2013', 'POPESTIMATE2014' : '2014', 'POPESTIMATE2015' : '2015', 'POPESTIMATE2016' : '2016', 'POPESTIMATE2017' : '2017', 'POPESTIMATE2018' : '2018'}
us_pop_2.rename(columns = us_pop_2_dict, inplace=True)
us_pop_2 = us_pop_2.transpose()
headers_7 = us_pop_2.iloc[0]
us_pop_2.rename(columns = headers_7, inplace=True)
us_pop_2 = us_pop_2.iloc[1:]
us_pop_2 = us_pop_2.reset_index()
us_pop_2.rename(columns = {'index' : 'Year'}, inplace = True)

In [15]:
# merge pop datasets

us_pop = us_pop_1.append(us_pop_2, ignore_index = True)

# convert column to string

us_pop['Year'] = us_pop['Year'].astype(str)
us_pop['Year'] = us_pop['Year'].str.replace("\.0", "")

__Note on U.S. population numbers:__ I use the official census numbers, released in April 2000 and April 2010. For non-census years, I use the population estimates, which are collected in July.

In [16]:
# merge all datasets 

us1 = pd.merge(us_lpr, us_labor, on = 'Year', how='left')
us2 = pd.merge(us1, us_ref_asy, on ='Year', how = 'left')
us = pd.merge(us2, us_pop, on='Year', how='left')
us = us.iloc[194:212]
us = us.set_index('Year')

In [17]:
# create smaller dataframe for graph

us_graph_1 = us[['Number LPR', 'Labor Migrants', 'Number of Admitted Refugees']].copy()
us_graph_1.reset_index(inplace=True)

In [18]:
# fix layout 

headers_8 = us_nonimm_country_17.iloc[2]
us_nonimm_country_17 = us_nonimm_country_17.iloc[3:]
us_nonimm_country_17.rename(columns=headers_8, inplace=True)
us_nonimm_country_17.columns = us_nonimm_country_17.columns.str.replace('United Kingdom9', 'United Kingdom')
us_nonimm_country_17.columns = us_nonimm_country_17.columns.str.replace('Australia1', 'Australia')

In [19]:
# read in dataset

country_codes_original = pd.read_excel(r'C:\Users\t_gas\Desktop\Political Economy - Masters\WS\Work, Labor, Social Reproduction\Term Paper\Data\Country Codes.xlsx')

# create dictionary

country_codes_us = dict(zip(country_codes_original['Country or Area'], country_codes_original['ISO-alpha3 Code']))

In [20]:
us_nonimm_country_map = us_nonimm_country_17.iloc[0]
us_nonimm_country_map = us_nonimm_country_map.iloc[3:]
us_nonimm_country_map = pd.DataFrame(us_nonimm_country_map)
us_nonimm_country_map= us_nonimm_country_map.reset_index()
us_nonimm_country_map.columns = ['Countries', 'Total']
us_nonimm_country_map['Code'] = us_nonimm_country_map['Countries'].map(country_codes_us)
us_nonimm_country_map['Total'] = pd.to_numeric(us_nonimm_country_map['Total'], errors='coerce')

In [21]:
# to identify errors, I used: us_nonimm_country_map.loc[us_nonimm_country_map['Code'].isna() == True]

In [22]:
us_nonimm_country_map['Countries'] = us_nonimm_country_map['Countries'].str.replace('Morocco6', 'Morocco')
us_nonimm_country_map['Countries'] = us_nonimm_country_map['Countries'].str.replace('Netherlands7', 'Netherlands')
us_nonimm_country_map['Countries'] = us_nonimm_country_map['Countries'].str.replace('New Zealand8', 'New Zealand')
us_nonimm_country_map['Countries'] = us_nonimm_country_map['Countries'].str.replace('Denmark4', 'Denmark')
us_nonimm_country_map['Countries'] = us_nonimm_country_map['Countries'].str.replace('France5', 'France')
us_nonimm_country_map['Countries'] = us_nonimm_country_map['Countries'].str.replace("China, People's Republic2,3", "China, People's Republic")

# create dictionary with errors

fix_us = { 
    "Bolivia (Plurinational State of)" : "Bolivia", 
    "Brunei Darussalam" : "Brunei", 
    "Myanmar" :  "Burma", 
    "China" : "China, People's Republic",
    "Democratic Republic of the Congo" : "Congo, Democratic Republic", 
    "Congo" : "Congo, Republic", 
    "Côte d’Ivoire" : "Cote d'Ivoire",  
    "Iran (Islamic Republic of)" : "Iran", 
    "Democratic People's Republic of Korea" : "Korea, North",
    "Republic of Korea" : "Korea, South",
    "Lao People's Democratic Republic": "Laos",
    "North Macedonia" : "Macedonia", 
    "Micronesia (Federated States of)" : "Micronesia, Federated States", 
    "Republic of Moldova" : "Moldova",  
    "Russian Federation" : "Russia", 
    "Serbia" : "Serbia and Montenegro (former)", 
    "Syrian Arab Republic" : "Syria", 
    "United Republic of Tanzania" : "Tanzania", 
    "United Kingdom of Great Britain and Northern Ireland" : "United Kingdom", 
    "Venezuela (Bolivarian Republic of)" : "Venezuela", 
    "Viet Nam" : "Vietnam"
    }


# fix values

country_codes_fixed_us = country_codes_original.copy()
country_codes_fixed_us['Country or Area'] = country_codes_original['Country or Area'].map(fix_us).fillna(country_codes_original['Country or Area'])


# override dictionary

country_codes_us = dict(zip(country_codes_fixed_us['Country or Area'], country_codes_fixed_us['ISO-alpha3 Code']))

In [23]:
us_nonimm_country_map['Code'] = us_nonimm_country_map['Countries'].map(country_codes_us)

# to check for existing errors, I used: us_nonimm_country_map.loc[us_nonimm_country_map['Code'].isna() == True]

In [24]:
# fix final values 

us_nonimm_country_map.loc[us_nonimm_country_map['Countries'] == "Kosovo", 'Code'] = "XKX"
us_nonimm_country_map.loc[us_nonimm_country_map['Countries'] == "Swaziland", 'Code'] = "SWZ"
us_nonimm_country_map.loc[us_nonimm_country_map['Countries'] == "Taiwan", 'Code'] = "TWN"
us_nonimm_country_map.loc[us_nonimm_country_map['Countries'] == "China, People's Republic", 'Code'] = "CHN"
us_nonimm_country_map.drop(us_nonimm_country_map.loc[us_nonimm_country_map['Countries'] == 'Unknown'].index, inplace=True)

# to check that no Nan values remain, I used: us_nonimm_country_map.loc[us_nonimm_country_map['Code'].isna() == True]

In [25]:
# fix layout

us_lpr_country_17.head(15)
us_lpr_country = us_lpr_country_17.iloc[14:, :2]
us_lpr_country.columns = ['Countries', 'Total']

fix_us_2 = {'D': '0', '-' : '0'}

us_lpr_country['Total'] = us_lpr_country['Total'].map(fix_us_2).fillna(us_lpr_country['Total'])
us_lpr_country['Total'].astype(float)

# merge dataframes

us_country_map = pd.merge(us_nonimm_country_map, us_lpr_country, on = 'Countries', how = 'left')
us_country_map.rename(columns = {'Total_x' : 'Nonimmigrants', 'Total_y' : 'Immigrants'}, inplace=True)
us_country_map = us_country_map[['Countries', 'Code', 'Nonimmigrants', 'Immigrants']]
us_country_map['Total'] = us_country_map[['Nonimmigrants', 'Immigrants']].astype(float).sum(axis=1)
us_country_map['Text'] = us_country_map['Countries'] + ': ' + us_country_map['Total'].astype(str)

## Germany Data

In [26]:
# fix layout

de_imm_number_clean = de_imm_number.iloc[:, [0, 3]]
de_imm_number_clean.columns = ['Visa', 'Total']

# create smaller dataframe

de_imm_number_visas = de_imm_number_clean.loc[
                    (de_imm_number_clean['Visa'] == 'Permission to reside') | 
                    (de_imm_number_clean['Visa'] == 'Temp. residence title according to Foreigners Act') |
                    (de_imm_number_clean['Visa'] == 'Unlim. residence title according to Foreigners Act') |
                    (de_imm_number_clean['Visa'] == 'Right of unlim. res. according to Foreigners Act') |
                    (de_imm_number_clean['Visa'] == 'Residence title for spec. purp. acc. to For. Act') |
                    (de_imm_number_clean['Visa'] == 'Residence title for except. purp. acc. to For. Act') |
                    (de_imm_number_clean['Visa'] == '1998-12-31') | 
                    (de_imm_number_clean['Visa'] == '1999-12-31') | 
                    (de_imm_number_clean['Visa'] == '2000-12-31') | 
                    (de_imm_number_clean['Visa'] == '2001-12-31') | 
                    (de_imm_number_clean['Visa'] == '2002-12-31') | 
                    (de_imm_number_clean['Visa'] == '2003-12-31') | 
                    (de_imm_number_clean['Visa'] == '2004-12-31') | 
                    (de_imm_number_clean['Visa'] == '2005-12-31') | 
                    (de_imm_number_clean['Visa'] == '2006-12-31') | 
                    (de_imm_number_clean['Visa'] == '2007-12-31') | 
                    (de_imm_number_clean['Visa'] == '2008-12-31') |
                    (de_imm_number_clean['Visa'] == '2009-12-31') |
                    (de_imm_number_clean['Visa'] == '2010-12-31') |
                    (de_imm_number_clean['Visa'] == '2011-12-31') |
                    (de_imm_number_clean['Visa'] == '2012-12-31') |
                    (de_imm_number_clean['Visa'] == '2013-12-31') |
                    (de_imm_number_clean['Visa'] == '2014-12-31') |
                    (de_imm_number_clean['Visa'] == '2015-12-31') |
                    (de_imm_number_clean['Visa'] == '2016-12-31') |
                    (de_imm_number_clean['Visa'] == '2017-12-31') 
                    ]

# reshape dataframe using .loc and lists

temp_res_df = de_imm_number_visas.loc[de_imm_number_visas['Visa'] == 'Temp. residence title according to Foreigners Act']
temp_res = temp_res_df['Total'].tolist()

unlim_res_df = de_imm_number_visas.loc[de_imm_number_visas['Visa'] == 'Unlim. residence title according to Foreigners Act']
unlim_res = unlim_res_df['Total'].tolist()

years_1 = ['1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017']

In [27]:
de_res = pd.DataFrame(columns=['Year',  
                           'Temp. residence title according to Foreigners Act', 
                           'Unlim. residence title according to Foreigners Act', 
                            ])

de_res['Year'] = years_1
de_res['Temp. residence title according to Foreigners Act'] = temp_res
de_res['Unlim. residence title according to Foreigners Act'] = unlim_res

In [28]:
# fix layout

de_ref_asy_hum = de_ref_asy_hum_number.iloc[:, [0, 3]]
de_ref_asy_hum = de_ref_asy_hum.drop([0, 1, 2, 3])
de_ref_asy_hum.columns = ['Visa', 'Total']

# reshape dataframe using .loc and lists

asylees_df = de_ref_asy_hum.loc[de_ref_asy_hum['Visa'] == 'S.25(1),RA,resid.permit,persons entitled to asylum']
asylees = asylees_df['Total'].tolist()

refugees_df = de_ref_asy_hum.loc[de_ref_asy_hum['Visa'] == 'S.25(2),RA,resid.permit,foreig.with refugee status']
refugees = refugees_df['Total'].tolist()

humanitarian_df = de_ref_asy_hum.loc[de_ref_asy_hum['Visa'] == 'S.22,1.s.,RA,res.perm., int.law,urg.human.grounds']
humanitarian = humanitarian_df['Total'].tolist()

years_2 = ['2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017']

de_prot = pd.DataFrame(columns=['Year', 
                           'Asylum', 
                           'Refugee', 
                           'Humanitarian', 
                            ])

de_prot['Year'] = years_2
de_prot['Asylum'] = asylees
de_prot['Refugee'] = refugees
de_prot['Humanitarian'] = humanitarian

In [29]:
de_pop = de_pop.iloc[4:22]
de_pop.columns = ['Year', 'Total']
years_3 = ['2000', '2001', '2002',  '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017']
de_pop['Year'] = years_3

In [30]:
# merge dataframes

de_1 = pd.merge(de_res, de_prot, on = 'Year', how = 'left')
de = pd.merge(de_1, de_pop, on='Year', how = 'left')
de.fillna("", inplace=True)
de['All Protected Persons'] = de[['Asylum', 'Refugee', 'Humanitarian']].sum(axis=1)

In [31]:
# fix layout

de_imm_country_map = de_imm_country_17.iloc[8:, [0, 3]]
de_imm_country_map.reset_index(inplace=True)
de_imm_country_map = de_imm_country_map.iloc[:, [1,2]]
de_imm_country_map = de_imm_country_map.iloc[:206, :]
de_imm_country_map.columns = ['Countries', 'Total']

In [32]:
# create dictionary 

country_codes_de = dict(zip(country_codes_original['Country or Area'], country_codes_original['ISO-alpha3 Code']))

# map dictionary 

de_imm_country_map['Code'] = de_imm_country_map['Countries'].map(country_codes_de)

# to check for errors, I used: de_imm_country_map.loc[de_imm_country_map['Code'].isna() == True]

In [33]:
# create dictionary with errors 

fix_de = { 
    "Afghanistan" : "Afghanistan, Republic of", 
    "Bolivia (Plurinational State of)" : "Bolivia, Plurinational State", 
    "Côte d’Ivoire" : "Cote d'Ivoire",
    "Iran (Islamic Republic of)" : "Iran, Islamic Republic of", 
    "Democratic Republic of the Congo" : "Congo, The Democratic Republic of the", 
    "Congo" : "Congo, Republic of the",
    "Democratic People's Republic of Korea" : "Korea, Democratic People's Republic",
    "Republic of Korea" : "Korea, Republic of",  
    "North Macedonia" : "Macedonia, the former Yugoslav Republic of",
    "Micronesia (Federated States of)" : "Micronesia, Federated States of",
    "Republic of Moldova" : "Moldova, Republic of",
    "Montenegro" : "Montenegro (since 2006-06-03)",
    "Serbia" : "Serbia (without Kosovo) (since 2008-02-17)",
    "Sudan" : "Sudan (including South Sudan) (until 2011-07-08)",
    "Syrian Arab Republic" : "Syrian",
    "United Republic of Tanzania" : "Tanzania", 
    "Holy See" : "Vatican City State", 
    "Venezuela (Bolivarian Republic of)" : "Venezuela, Bolivarian Republic of", 
    "United States of America" : "United States", 
    "United Kingdom of Great Britain and Northern Ireland" : "United Kingdom"
    }


# fix values

country_codes_fixed_de = country_codes_original.copy()
country_codes_fixed_de['Country or Area'] = country_codes_original['Country or Area'].map(fix_de).fillna(country_codes_original['Country or Area'])


# override dictionary

country_codes_de = dict(zip(country_codes_fixed_de['Country or Area'], country_codes_fixed_de['ISO-alpha3 Code']))

In [34]:
# map new dictionary 

de_imm_country_map['Code'] = de_imm_country_map['Countries'].map(country_codes_de)

# to check for errors, I used: de_imm_country_map.loc[de_imm_country_map['Code'].isna() == True]

In [35]:
# fix final values

de_imm_country_map.loc[de_imm_country_map['Countries'] == "China", 'Code'] = "CHN"
de_imm_country_map.loc[de_imm_country_map['Countries'] == "United Kingdom", 'Total'] = 117250 
de_imm_country_map.drop(de_imm_country_map.loc[de_imm_country_map['Countries'] == 'British Overseas Territories'].index, inplace=True)
de_imm_country_map.loc[de_imm_country_map['Countries'] == "Hong Kong", 'Code'] = "HKG"
de_imm_country_map.drop(de_imm_country_map.loc[de_imm_country_map['Countries'] == 'Yugoslavia, Socialist Federal Rep. (u. 1992-04-26)'].index, inplace=True)
de_imm_country_map.drop(de_imm_country_map.loc[de_imm_country_map['Countries'] == 'Yugoslavia, Federal Rep.(1992-04-27 to 2003-02-04)'].index, inplace=True)
de_imm_country_map.loc[de_imm_country_map['Countries'] == "Kosovo", 'Code'] = "XKX"
de_imm_country_map.loc[de_imm_country_map['Countries'] == "Macau", 'Code'] = "MAC"
de_imm_country_map.loc[de_imm_country_map['Countries'] == "Palestinian Territories", 'Code'] = "PSE"
de_imm_country_map.loc[de_imm_country_map['Countries'] == "Serbia", 'Total'] = 251915
de_imm_country_map.drop(de_imm_country_map.loc[de_imm_country_map['Countries'] == 'Serbia (incl. Kosovo) (2006-06-03 to 2008-02-16)'].index, inplace=True)
de_imm_country_map.drop(de_imm_country_map.loc[de_imm_country_map['Countries'] == 'Serbia and Montenegro (2003-02-05 to 2006-06-02)'].index, inplace=True)
de_imm_country_map.loc[de_imm_country_map['Countries'] == "Russian Federation", 'Total'] = 254050
de_imm_country_map.drop(de_imm_country_map.loc[de_imm_country_map['Countries'] == 'Soviet Union (until 1991-12-25)'].index, inplace=True)
de_imm_country_map.loc[de_imm_country_map['Countries'] == "Sudan", 'Total'] = 7760
de_imm_country_map.drop(de_imm_country_map.loc[de_imm_country_map['Countries'] == 'Sudan (without South Sudan) (since 2011-07-09)'].index, inplace=True)
de_imm_country_map.loc[de_imm_country_map['Countries'] == "South Sudan (since 2011-07-09)", 'Code'] = "SSD"
de_imm_country_map.loc[de_imm_country_map['Countries'] == "Swaziland", 'Code'] = "SWZ"
de_imm_country_map.loc[de_imm_country_map['Countries'] == "Taiwan", 'Code'] = "TWN"
de_imm_country_map.drop(de_imm_country_map.loc[de_imm_country_map['Countries'] == 'Czechoslovakia (until 1992-12-31)'].index, inplace=True)
de_imm_country_map.loc[de_imm_country_map['Total'] == "-"] = 0

In [36]:
# as a final check, I used: de_imm_country_map.loc[de_imm_country_map['Code'].isna() == True]

In [37]:
# create Text column 

de_imm_country_map['Text'] = de_imm_country_map['Countries'].astype(str) + ": " + de_imm_country_map['Total'].astype(str)

In [38]:
# to more easily compare U.S. and Germany, I merged dataframes

comp = pd.merge(de, us, on = 'Year', how = 'left')
comp.drop(columns = ['Asylum', 'Refugee', 'Humanitarian', 'Annual Ceiling'], inplace=True)
comp.drop([0, 1], axis = 0, inplace = True)
comp.columns = ['Year', 'DE: Temp Res', 'DE: Perm Res', 'DE: Pop', 'DE: Prot', 'US: Perm Res', 'US: Temp Res', 'US: Prot', 'US: Pop']
comp = comp[['Year', 'DE: Temp Res', 'DE: Perm Res', 'DE: Prot', 'DE: Pop', 'US: Temp Res', 'US: Perm Res', 'US: Prot', 'US: Pop']]
comp = comp.reset_index(drop = True)
comp['DE: % Temp'] = (comp['DE: Temp Res'] / comp['DE: Pop']) * 100
comp['DE: % Perm'] = (comp['DE: Perm Res'] / comp['DE: Pop']) * 100
comp.loc[comp['DE: Prot'] == "", 'DE: Prot'] = 0
comp['DE: % Prot'] = (comp['DE: Prot'] / comp['DE: Pop']) * 100
comp['US: % Temp'] = (comp['US: Temp Res'] / comp['US: Pop']) * 100
comp['US: % Perm'] = (comp['US: Perm Res'] / comp['US: Pop']) * 100
comp['US: % Prot'] = (comp['US: Prot'] / comp['US: Pop']) * 100
comp = comp[['Year', 'DE: Temp Res', 'DE: % Temp', 'DE: Perm Res', 'DE: % Perm', 'DE: Prot', 'DE: % Prot', 'DE: Pop', 'US: Temp Res', 'US: % Temp', 'US: Perm Res', 'US: % Perm', 'US: Prot', 'US: % Prot', 'US: Pop']]
comp.loc[comp['DE: % Prot'] == 0, 'DE: % Prot'] = "No Data"

In [40]:
# export all dataframes to .csv files 

us_graph_1.to_csv("us_graph_1.csv")
de.to_csv("de.csv")
comp.to_csv("comp.csv")
us_country_map.to_csv("us_country_map.csv")
de_imm_country_map.to_csv("de_imm_country_map.csv")